- Retention Rate aka коэффициент удержания
- Churn Rate aka коэффициент отскока-
- CR aka conversion rate
- LTV aka Lifetime Value aka пожизненная ценность
- CAC aka Customer Acquisition Cost aka стоимость привлечения клиента
- ROI aka Return On Investment aka возврат на инвестиции: LTV / CAC
- ROMI aka Return On Marketing Investment
- ARPU aka Average Revenue Per User aka средняя выручка с пользователя
- ARPPU aka Average Revenue Per Paying User aka средняя выручка с платящего пользователя
- DAU aka daily active users
- WAU aka weekly active users
- MAU aka monthly active users

In [1]:
import pandas as pd
import json

In [6]:
links = {
    'csv': [
        '../db.tst/yandex_music_project.csv',
        'https://code.s3.yandex.net/datasets/ad_data.csv',
        'https://code.s3.yandex.net/datasets/site_data.csv'
    ],
    'json': [
        '../../.dts/tmp/e1.json'
    ],
    'urls': [
        'https://www.w3schools.com/html/html_tables.asp'
    ]
}

In [13]:
t_ad = pd.read_csv(links['csv'][1])

In [14]:
t_site = pd.read_csv(links['csv'][2])

In [10]:
df = pd.read_html(links['urls'][0])

In [5]:
df = pd.read_json(links['json'][0], orient='index')

# orient= [
#     'index',
#     'columns',
#     'split',
#     'records'
#     ]

In [ ]:
# save to json
#df.to_json('example.json')

In [15]:
funnel = pd.merge(t_ad, t_site, on='date')

In [18]:
t_ad.sample(2)

,date,impressions,clicks
12,2019-08-13,9682,78
26,2019-08-27,7878,73


In [30]:
funnel.head(5)
#df.loc['info']

,date,impressions,clicks,registrations,"ctr, %","cr, %",week,month
0,2019-08-01,6153,61,6,0.991386,9.836066,31,8
1,2019-08-02,8254,81,6,0.981342,7.407407,31,8
2,2019-08-03,4434,43,4,0.969779,9.302326,31,8
3,2019-08-04,9510,79,7,0.830705,8.860759,31,8
4,2019-08-05,7167,70,6,0.976699,8.571429,32,8


In [19]:
# CR aka conversion rate
# CTR aka click-through rate
funnel['ctr, %'] = (funnel['clicks']/funnel['impressions'])*100
funnel['cr, %'] = (funnel['registrations']/funnel['clicks'])*100

In [22]:
funnel['date'] = pd.to_datetime(funnel['date'])
funnel['week'] = funnel['date'].dt.isocalendar().week

In [29]:
funnel['month'] = funnel['date'].dt.month

In [26]:
funnel_ = funnel.groupby('week')[['registrations', 'clicks']].sum()
funnel_.sample(3)

,registrations,clicks
week,,
32,43,503
31,23,264
33,39,471


In [27]:
funnel_['registrations'] / funnel_['clicks'] * 100

week
31    8.712121
32    8.548708
33    8.280255
34    8.726415
35    8.143322
dtype: float64

In [23]:
funnel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 0 to 29
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           30 non-null     datetime64[ns]
 1   impressions    30 non-null     int64         
 2   clicks         30 non-null     int64         
 3   registrations  30 non-null     int64         
 4   ctr, %         30 non-null     float64       
 5   cr, %          30 non-null     float64       
 6   week           30 non-null     UInt32        
dtypes: UInt32(1), datetime64[ns](1), float64(2), int64(3)
memory usage: 1.8 KB


In [6]:
df.columns # duplicate columns from info()

AttributeError: 'list' object has no attribute 'columns'

In [7]:
# counts of unique values
print(df['artist'].value_counts().head(3)) 

Sasha     6
Plan B    6
Koala     6
Name: artist, dtype: int64


In [6]:
df = df.rename(columns={
    '  userID': 'user_id',
    'Track': 'track',
    '  City  ': 'city',
    'Day': 'day'
})

In [8]:
df.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

In [9]:
columns_to_replace = ['track', 'artist', 'genre']
for column in columns_to_replace:
    df[column] = df[column].fillna('unknown') 

In [12]:
df.duplicated().sum()

0

In [11]:
# duplicate resolving
df = df.drop_duplicates().reset_index(drop=True) 

In [5]:
#print(df['genre'].sort_values().unique()[:10])
print(df.sort_values('genre', ascending=False)[:3])

         userID            Track            artist        genre  \
6501     1D4661      Раз два три             Какао  электроника   
8514   A439123F          Flip It               NaN          ïîï   
28665  D7FB50DA  Drumming Circle  Professor Trance    worldbeat   

                 City        time        Day  
6501   Saint-Petersburg  08:54:15  Wednesday  
8514             Moscow  09:08:51     Friday  
28665            Moscow  09:30:47     Monday  


In [15]:
duplicates_genre = ['hip', 'hop', 'hip-hop']
df['genre'] = df['genre'].replace(duplicates_genre, 'hiphop')

In [16]:
df.groupby('city')['track'].count()

city
Moscow              42741
Saint-Petersburg    18512
Name: track, dtype: int64

In [5]:
# sorting methods

print(df.sort_values('artist')[:3])

         userID            Track   artist       genre            City    \
40006  30B60957         Big City   Gus G.       metal            Moscow   
30942  176A5342         Классика    Jenee      rusrap            Moscow   
64600  A1C42156  Hold On To Love   Netsky  electronic  Saint-Petersburg   

           time     Day  
40006  20:13:26  Friday  
30942  09:18:18  Monday  
64600  08:16:39  Monday  
